In [ ]:
!pip install -q transformers pdf2image torch torchvision pillow pytesseract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 763.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.0 MB/s eta 0:00:00


In [ ]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 186 kB in 1s (309 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.6_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.6) ...
Setting up poppler-utils (22.02.0-2ubuntu0.6) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
import os
import torch
from transformers import AutoModelForObjectDetection, AutoProcessor
from pdf2image import convert_from_path
from PIL import Image
from google.colab import files

pdf_path = "/content/ESG-Research-Controversies-Methodology.pdf"

# Convert PDF pages to images
images = convert_from_path(pdf_path, dpi=400)

# Load Hugging Face Table Transformer model
model_name = "microsoft/table-transformer-detection"
model = AutoModelForObjectDetection.from_pretrained(model_name)
processor = AutoProcessor.from_pretrained(model_name)

# Create output directory
output_dir = "/content/extracted_tables"
os.makedirs(output_dir, exist_ok=True)

# Process each page
for page_num, image in enumerate(images):
    inputs = processor(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)

    # Detect tables with a confidence threshold
    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

    # Debugging: Print number of detected tables
    print(f"Page {page_num + 1}: {len(results['boxes'])} tables detected")

    # Extract and save only the table regions
    for idx, box in enumerate(results["boxes"]):
        box = [round(i, 2) for i in box.tolist()]
        x1, y1, x2, y2 = map(int, box)  # Convert to integer for cropping

        # Crop the detected table
        table_crop = image.crop((x1, y1, x2, y2))

        # Save the cropped table image
        output_path = os.path.join(output_dir, f"table_page_{page_num + 1}_table_{idx + 1}.png")
        table_crop.save(output_path)
        print(f"Saved table: {output_path}")

# Zip and Download Extracted Tables
!zip -r extracted_tables.zip extracted_tables
files.download("extracted_tables.zip")


Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Page 1: 0 tables detected
Page 2: 0 tables detected
Page 3: 0 tables detected
Page 4: 0 tables detected
Page 5: 0 tables detected
Page 6: 1 tables detected
Saved table: /content/extracted_tables/table_page_6_table_1.png
Page 7: 0 tables detected
Page 8: 1 tables detected
Saved table: /content/extracted_tables/table_page_8_table_1.png
Page 9: 0 tables detected
Page 10: 0 tables detected
Page 11: 1 tables detected
Saved table: /content/extracted_tables/table_page_11_table_1.png
Page 12: 0 tables detected
Page 13: 0 tables detected
Page 14: 0 tables detected
Page 15: 0 tables detected
Page 16: 0 tables detected
Page 17: 0 tables detected
Page 18: 1 tables detected
Saved table: /content/extracted_tables/table_page_18_table_1.png
Page 19: 0 tables detected
Page 20: 1 tables detected
Saved table: /content/extracted_tables/table_page_20_table_1.png
Page 21: 1 tables detected
Saved table: /content/extracted_tables/table_page_21_table_1.png
Page 22: 0 tables detected
Page 23: 0 tables detected


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>